### Cursor Extensions

1. From the View menu, select Extensions
2. Search for Python
3. Click on "Python" made by "ms-python" and select Install if not already installed
4. Search for Jupyter
5. Click on "Jupyter" made by "ms-toolsai" and select Install of not already installed


### Next Select the Kernel

Click on "Select Kernel" on the Top Right

Choose "Python Environments..."

Then choose the one that looks like `.venv (Python 3.12.x) .venv/bin/python` - it should be marked as "Recommended" and have a big star next to it.

Any problems with this? Head over to the troubleshooting.

### Note: you'll need to set the Kernel with every notebook..

In [ ]:
# imports
# using cursor 2.0
# I had to install the openai package and the dotenv package and beatiful soup package and requests package
# and scraper package

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  
OPENAI_API_KEY=
GEMINI_API_KEY=

In [7]:
# Load environment variables in a file called .env -- ls -a

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [8]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [ ]:
openai = OpenAI()
# as of 11/2/2025, gpt-5-nano is the latest and the cheapest model

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Nice to meet you. Thanks for saying hi.\n\nWhat would you like to do today? I can help with many things, like:\n- Explain topics in simple terms\n- Help with writing (emails, stories, resumes)\n- Plan trips, events, or study schedules\n- Debug or brainstorm code\n- Learn or practice a language\n- Brainstorm ideas or generate creative content (poems, jokes, prompts)\n\nIf you’re not sure, tell me a topic you’re curious about or a task you have in mind, and I’ll suggest a starting point.'

## OK onwards with our first project

In [11]:
# Let's try out this utility

anthro = fetch_website_contents("https://anthropic.com")
print(anthro)

Home \ Anthropic

Skip to main content
Skip to footer
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsible Scaling Policy
Trust center
Security and compliance
Learn
Learn
Anthropic Academy
Engineering at Anthropic
Developer docs
Company
About
Careers
Events
News
Try Claude
Try Claude
Try Claude
Learn more about Claude
Products
Claude
Claude Code
Claude Developer Platform
Pricing
Contact sales
Models
Opus
Sonnet
Haiku
Log in
Claude.ai
Claude Console
EN
This is some text inside of a div block.
Log in to Claude
Log in to Claude
Log in to Claude
Download app
Download app
Download app
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsible Scaling Policy
Trust center
Security and compliance
Learn
Learn
Anthropic Academy
Engineering at Anthropic
Developer docs
Company
About
Careers
Events
News
Try Claude
Try Claude
Try Claude
Learn more about Claude
Products
Claude
Claude Code
Claude Developer Platform
Pricing
Contact sales
Models
Opus
Sonnet
Haiku
Lo

## Types of prompts

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [12]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [13]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [16]:
# Try this out, and then try for a few more websites

messages_for(anthro)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome \\ Anthropic\n\nSkip to main content\nSkip to footer\nResearch\nEconomic Futures\nCommitments\nInitiatives\nTransparency\nResponsible Scaling Policy\nTrust center\nSecurity and compliance\nLearn\nLearn\nAnthropic Academy\nEngineering at Anthropic\nDeveloper docs\nCompany\nAbout\nCareers\nEvents\nNews\nTry Claude\nTry Claude\nTry Claude\nLearn more about Claude\nProducts\nClaude\nClaude Code\nClaude Developer Platform\nPricing\nContact sales\nModels\nOpus\nSonnet\nHaiku\nLog in\nClaude.ai\nClaude Console\nEN\

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!
# scraper.feth_website_contents is a function that fetches the contents of a website

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-5-nano",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [18]:
summarize("https://anthropic.com")

'- Anthropic positions itself as a safety-forward AI lab turned public-benefit company, focused on long-term well-being and risk mitigation rather than just pushing shiny capabilities.\n\n- Product lineup mentioned: Claude family (Claude, Claude Code) plus specialized specials like Claude Sonnet 4.5 (claimed as the best for agents, coding, and computer use) and Claude Haiku 4.5 (noted for managing context on the Claude Developer Platform).\n\n- Core themes you’ll encounter: a heavy emphasis on safety, responsible scaling, transparency, trust, and security/compliance. They tease governance and policy work alongside their research and products.\n\n- Offerings beyond models: a big ecosystem including Research, Learn (Anthropic Academy, developer docs), Developer Platform, pricing, and a Trust/Transparency/Policy focus. News and Events sections are present too.\n\n- News/announcements: there are explicit model announcements for Claude Sonnet 4.5 and Claude Haiku 4.5, with “Read announcemen

In [19]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [20]:
display_summary("https://anthropic.com")

- Snarky gist: It’s Anthropic’s marketing hub for safety-forward AI, loudly pitching the Claude family, a splash of corporate goodness, and a couple of model announcements. Translation: “We build safe AI and tell you about it a lot.”

- Core mission: They frame AI as having huge impact and frame themselves as a public-benefit corporation dedicated to securing benefits and mitigating risks. Lots of ethos about long-term human well-being and responsible progress.

- Product lineup you’ll care about: Claude family is front and center, with Loveable names like Claude Sonnet 4.5 (their claim: best for agents, coding, and computer use) and Claude Haiku 4.5 (focus on managing context on the Claude Developer Platform). Also references to Claude, Claude Code, Claude Developer Platform, Claude.ai, and Claude Console.

- Announcements you’ll actually see: Two “Read announcement” prompts for 4.5 updates (Sonnet 4.5 and Haiku 4.5), plus a mission-style blurb that AI safety is their north star.

- Other not-quite-news-but-helps-cred: A stack of sections like Research, Transparency, Security & Compliance, Trust Center, and Responsible Scaling Policy—basically the safety-and-governance boilerplate they lean on.

- Vibe and extra bits: Lots of nav repetition, “This is some text inside a div block” filler, and a heavy dose of “Try Claude” CTAs. The core message is: safety-first AI with a toolbox of Claude variants, aimed at developers and enterprise users.

If you want the elevator pitch: Anthropic sells the Claude AI family and safety-first messaging, rides on model updates (4.5), and packages governance/go-to-market fluff as their differentiator.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [21]:
display_summary("https://mistral.ai")

- Snarky summary: It's basically a marketing splash for Mistral AI's Frontier AI. Claims you can train, distill, fine-tune, and deploy enterprise-grade AI anywhere while keeping data private. It teases a single multilingual, multimodal assistant that can search, create, code, learn, automate, and collaborate. Add in “le Chat,” AI Studio, Mistral Code, and expert-led services, and you’ve got a one-stop shop for customizable AI that supposedly works from edge to cloud—plus lots of buzzwords about safety and hands-on support.

- What it’s selling (key offerings):
  - Configurable, deployable-anywhere AI platform with fine-tune and distillation capabilities
  - Enterprise-grade tooling and agent-ready deployments with safety features
  - Privacy-first approach (on-prem, cloud, edge, devices)
  - Hands-on, expert-led deployment and solutioning
  - Unified AI assistant with multi-language and multi-modal support
  - Specific products/services teased: le Chat, AI Studio, Mistral Code, and specialized services

- News or announcements: None detected in the provided text. It’s all marketing copy and product mentions, no dates, press releases, or news items.

- Quick takeaway if you’re evaluating: Expect heavy emphasis on configurability, enterprise deployment options, and a bundled suite (coding aid, chat/assistant, and specialized AI acceleration) with a lot of marketing speak about privacy and hands-on support. But there aren’t concrete specs or real-time updates in this snippet.

In [22]:
display_summary("https://perplexity.ai")

- Summary: The site is currently blocked behind a “Just a moment…” gate that asks you to enable JavaScript and cookies. There’s no actual content to read, so it’s basically a door with a bouncer, not a page to summarize.
- News/Announcements: None accessible or visible. If there were any, they aren’t loading behind the gate yet.
- Takeaway: If you want me to summarize real content, you’ll need to load the page properly (enable JS/cookies) or paste the content you want summarized.

In [25]:
# Step 1: Create your prompts

system_prompt = "you are a parent to a high school student who is trying to responds to a email from a counselor at school."
user_prompt = f"""
    It is complicated and hard to understand how to calculate the GPA of a student with weighted classes and unweighted classes and AP classes and etc.
    Respond to the email from the counselor asking how to calculate the GPA with a clear and concise explanation.
    The email is as follows:
    "Greetings, Hawk Families!
This is just a reminder of our live Webex meeting with our counselors today at 12 noon. See details below and flyer attached to join us for the meeting today:
Transcripts, Grades, & GPA Live Parent Meeting - Thursday, October 30
The H9 Counselors will go live via Webex today at noon.  Parents are invited to join in for this informative meeting as the counselors talk about the different components of the transcript including GPA and credits.  The counselors will allow for a Q&A time as well. See flyer attached - Virtual Parent Meeting - for details.
"""

# Step 2: Make the messages list

messages = messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI
response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)

# Step 4: print the result
print(response.choices[0].message.content)

Subject: Re: Transcripts, Grades, & GPA Live Parent Meeting

Hello [Counselor’s Name] and H9 Counseling Team,

Thank you for the reminder about today’s Webex. We plan to join at noon.

To keep things clear, here’s a concise outline of how GPA is typically calculated, and I’d appreciate confirmation of our school’s specifics during the meeting:

- Unweighted GPA (4.0 scale): convert each course grade to points (A=4.0, A-=3.7, B+=3.3, B=3.0, etc.), multiply by course credits, sum, then divide by total credits.

- Weighted GPA (for Honors/AP): most commonly add a course weight (e.g., Honors +0.5, AP +1.0) to the base grade points (or use a school-specific weighted scale), then multiply by credits, sum, and divide by total credits.

- Notes: Some schools report two GPAs (unweighted and weighted). Policies on repeats, withdrawals, and credits can vary.

Could you please confirm:
- The exact unweighted and weighted scales our school uses and how weights are applied?
- The credits assigned to

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)